In [2]:
from __future__ import print_function
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
import os

batch_size = 32
num_classes = 10
epochs = 100
data_augmentation = True
num_predictions = 20
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'keras_cifar10_trained_model.h5'

# The data, split between train and test sets:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

# initiate RMSprop optimizer
opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)

# Let's train the model using RMSprop
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

if not data_augmentation:
    print('Not using data augmentation.')
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True)
else:
    print('Using real-time data augmentation.')
    # This will do preprocessing and realtime data augmentation:
    datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        zca_epsilon=1e-06,  # epsilon for ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        # randomly shift images horizontally (fraction of total width)
        width_shift_range=0.1,
        # randomly shift images vertically (fraction of total height)
        height_shift_range=0.1,
        shear_range=0.,  # set range for random shear
        zoom_range=0.,  # set range for random zoom
        channel_shift_range=0.,  # set range for random channel shifts
        # set mode for filling points outside the input boundaries
        fill_mode='nearest',
        cval=0.,  # value used for fill_mode = "constant"
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False,  # randomly flip images
        # set rescaling factor (applied before any other transformation)
        rescale=None,
        # set function that will be applied on each input
        preprocessing_function=None,
        # image data format, either "channels_first" or "channels_last"
        data_format=None,
        # fraction of images reserved for validation (strictly between 0 and 1)
        validation_split=0.0)

    # Compute quantities required for feature-wise normalization
    # (std, mean, and principal components if ZCA whitening is applied).
    datagen.fit(x_train)

    # Fit the model on the batches generated by datagen.flow().
    model.fit_generator(datagen.flow(x_train, y_train,
                                     batch_size=batch_size),
                        steps_per_epoch=x_train.shape[0] // batch_size,
                        epochs=epochs,
                        validation_data=(x_test, y_test),
                        workers=4)

# Save model and weights
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
model_path = os.path.join(save_dir, model_name)
model.save(model_path)
print('Saved trained model at %s ' % model_path)

# Score trained model.
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples
Using real-time data augmentation.
Epoch 1/100
1562/1562 [==============================] - 180s 116ms/step - loss: 1.8884 - acc: 0.3029 - val_loss: 1.5643 - val_acc: 0.4403
Epoch 2/100
1562/1562 [==============================] - 179s 115ms/step - loss: 1.5922 - acc: 0.4152 - val_loss: 1.5046 - val_acc: 0.4576
Epoch 3/100
1562/1562 [==============================] - 176s 113ms/step - loss: 1.4703 - acc: 0.4688 - val_loss: 1.3040 - val_acc: 0.5281
Epoch 4/100
1562/1562 [==============================] - 176s 113ms/step - loss: 1.3884 - acc: 0.5000 - val_loss: 1.2682 - val_acc: 0.5593
Epoch 5/100
1562/1562 [==============================] - 172s 110ms/step - loss: 1.3175 - acc: 0.5289 - val_loss: 1.1915 - val_acc: 0.5860
Epoch 6/100
1562/1562 [==============================] - 174s 112ms/step - loss: 1.2579 - acc: 0.5512 - val_loss: 1.1087 - val_acc: 0.6075
Epoch 7/100
1562/1562 [==============================] - 1

1562/1562 [==============================] - 164s 105ms/step - loss: 0.7919 - acc: 0.7325 - val_loss: 0.6877 - val_acc: 0.7756
Epoch 59/100
1562/1562 [==============================] - 164s 105ms/step - loss: 0.7859 - acc: 0.7345 - val_loss: 0.7023 - val_acc: 0.7698
Epoch 60/100
1562/1562 [==============================] - 162s 104ms/step - loss: 0.7883 - acc: 0.7378 - val_loss: 0.6680 - val_acc: 0.7805
Epoch 61/100
1562/1562 [==============================] - 159s 102ms/step - loss: 0.7921 - acc: 0.7344 - val_loss: 0.7254 - val_acc: 0.7606
Epoch 62/100
1562/1562 [==============================] - 160s 102ms/step - loss: 0.7911 - acc: 0.7337 - val_loss: 0.6965 - val_acc: 0.7662
Epoch 63/100
1562/1562 [==============================] - 161s 103ms/step - loss: 0.7860 - acc: 0.7358 - val_loss: 0.7128 - val_acc: 0.7682
Epoch 64/100
1562/1562 [==============================] - 163s 104ms/step - loss: 0.7880 - acc: 0.7369 - val_loss: 0.7236 - val_acc: 0.7747
Epoch 65/100
1562/1562 [=========